In [ ]:
# import and run the poisson4 script
import poisson4
import eztfem as ezt

results = poisson4.main()

max_diff = results[0]
mesh     = results[1]
problem  = results[2]
u        = results[3]
derivs   = results[4]

In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)

In [ ]:
# plot the solution field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400))